# Second Module
# Optimization Module

# Step Zero

In [1]:
time_periods = list(range(int(input("Enter the start time period (e.g., 0): ").strip()), int(input("Enter the end time period (e.g., 72): ").strip()) + 1))
solar_capital_cost = int(input("Enter the solar installation capital cost for the year of the study (e.g., $1,032,000 / MW for 2021): ").strip())
wind_capital_cost = int(input("Enter the wind installation capital cost for the year of the study (e.g., $1,404,000/ MW for 2021): ").strip())

# Store the variables for use in other scripts
with open('good_model_working/user_inputs.py', 'w') as f:
    f.write(f"time_periods = {time_periods}\n")
    f.write(f"solar_capital_cost = {solar_capital_cost}\n")
    f.write(f"wind_capital_cost = {wind_capital_cost}\n")

# Import Packages

In [2]:
# import all necessary packages & modules
import importlib
import pyomo.environ as pyomo
import pickle
import good_model_working
from good_model_working import utils
from good_model_working.reload import deep_reload
import warnings
import logging
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=UserWarning)


# Creating Model's Graph

In [3]:
# Creating graph and sets for main module
deep_reload(good_model_working)

# import the model data & sets
input_sets = utils.get_sets('/Users/haniftayarani/good_model/Model_Input/all_input_sets_sorted.json')
model_sets = utils.filter_sets(input_sets, utils.gen_to_remove)
graph = utils.create_graph('/Users/haniftayarani/good_model/Model_Input/all_input_objects.json')

# Preparing Model's Input

In [4]:
# Selecting the region to run the ED
deep_reload(good_model_working)

user_input = input('Please select your region(s) for analysis (separated by commas): \nAll\nFlorida\nMISO\nNew England\nPJM\nSSP\nSoutheast\nERCOT\nWECC\n')
subgraph, sub_nodes = utils.get_subgraph(user_input, graph)
constraint_to_deactivate = ['policy'] if input("Do you want to include the RPS policy in this study? (yes/no): ").strip().lower() == 'no' else []
rps_policy_set = 0 if 'policy' in constraint_to_deactivate else (float(input("Enter the RPS policy percentage (e.g., 20 for 20%): ").strip()) / 100)
# constraint_to_deactivate = ['generator', 'solar', 'wind', 'storage', 'policy']
model_data = {
    'test_nodes': True,
    'test_cons': False,
    'graph': graph, 
    'sets': model_sets, 
    'subgraph_nodes': sub_nodes, 
    'subgraph': subgraph,
    'rps_policy': rps_policy_set,
    'constraint_deactivation': constraint_to_deactivate,
}

# Building the Optimization Model

In [5]:
# import good_model_working
deep_reload(good_model_working)

#Change this to point to your version of cbc or use another solver
solver_name='appsi_highs'

# run model 
problem = good_model_working.opt_model.Opt_Model(model_data, solver_name)

[+  47.26] Grid built
[+   0.00] Sets built
[+   1.32] Parameters built
Total demand for each region:
Region: ERC_REST, Total Demand: 26927663.358032044
Region: ERC_WEST, Total Demand: 2268474.9605390998
Region: FRCC, Total Demand: 17765826.046312336
Region: MIS_AMSO, Total Demand: 2701908.2397382073
Region: MIS_AR, Total Demand: 3172402.8692677924
Region: MIS_D_MS, Total Demand: 1892917.3522922893
Region: MIS_IA, Total Demand: 1863032.644964113
Region: MIS_IL, Total Demand: 4021985.554455995
Region: MIS_INKY, Total Demand: 8029957.471268379
Region: MIS_LA, Total Demand: 3856512.557586518
Region: MIS_LMI, Total Demand: 8289201.419806404
Region: MIS_MAPP, Total Demand: 746708.5561001198
Region: MIS_MIDA, Total Demand: 2560576.070385142
Region: MIS_MNWI, Total Demand: 7983824.790126631
Region: MIS_MO, Total Demand: 3531040.5803510314
Region: MIS_WOTA, Total Demand: 2813335.6453788765
Region: MIS_WUMS, Total Demand: 5454478.952043472
Region: NENGREST, Total Demand: 6741916.761849412
Regio

# Run the Optimization Model 

In [6]:
deep_reload(good_model_working.utils)

# Generate simple model statistics
utils.get_model_statistcs(problem.model)
utils.get_total_generator_count(problem.model)

# Solve the model
problem.solve_model()

# print the objective value
print(f"Total system cost: ${pyomo.value(problem.model.obj_func):,.2f}")

 total_variables  total_constraints
         3690127            3754296
Total generators: 4777.0
Running HiGHS 1.5.3 [date: 2023-05-16, git hash: 594fa5a9d]
Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
45199 rows, 3577330 cols, 5172434 nonzeros
45198 rows, 1990658 cols, 3541603 nonzeros
45198 rows, 1990658 cols, 3541603 nonzeros
Presolve : Reductions: rows 45198(-3709098); columns 1990658(-1699469); elements 3541603(-6934144)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     1.9981453027e+06 Pr: 41742(4.9786e+11) 5s
      82570     5.3559759729e+09 Pr: 16074(5.48198e+09); Du: 0(3.66741e-07) 10s
     110709     5.5075316296e+09 Pr: 0(0); Du: 0(1.11022e-16) 12s
     110709     5.5075316296e+09 Pr: 0(0); Du: 0(1.11022e-16) 12s
Solving the original LP from the solution after postsolve
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
  

# Saving the Model Output

In [7]:
deep_reload(good_model_working)

# Get model results
results = problem.get_results()
# Specify the path to save the pickle file
pickle_file_path = 'results.pickle'

# Save the dictionary as a pickle file
with open(pickle_file_path, 'wb') as f:
    pickle.dump(results, f)
    
nodes_dict = results.get('nodes', {})